## Week Five Class One: Allocation and Mix Problem (Desks, Tables, and Chairs)

## Table of Contents<a id="TOC"></a>

1. [Install Packages](#S1) <br>
2. [Allocation and Mix (Desks, Tables and Chairs)](#S2) <br>
5. [ERRORS](#S3) <br>

For our third example, let's examine the Allocation and Mix problem for Desks, Tables, and Chairs.


## 1. Install Packages<a id=S1></a>

In [2]:
import pandas as pd
import pyomo.environ as pe

##### [Back to Top](#TOC)

## 2. Allocation and Mix (Desks, Tables and Chairs)<a id=S2></a>

### Read in data from Excel

In [3]:
raw_data = pd.read_excel("w05-c01-covering-allocation.xlsx", sheet_name="Allocation and Mix")
raw_data

,Allocation: Product Mix,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Decision Variables,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,C,D,T,NaN,NaN,NaN,NaN
3,Production plan,1,1,1,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Objective Function,NaN,NaN,NaN,Total,NaN,NaN,NaN
6,Profit,16,20,14,50,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Constraints,NaN,NaN,NaN,LHS,NaN,RHS,slack
9,Fabrication,4,6,2,12,<=,2000,1988


Let's create the index names for the Decision Variables.

In [4]:
DV_indexes = ['chairs', 'desks', 'tables']

This next cell extracts the coefficients of the objective function (profit) and the LHS of the resource constraints. Remember that indexes start with 0. So the first row is 0 and the first column is 0. In addition, remember that ranges go "up to" the upper limit so 1:4 is 1,2,3.

In [22]:
coef = pd.DataFrame(raw_data.iloc[[6, 9, 10, 11, 12], 1:4])
coef.index = ['profit', 'fabrication', 'assembly', 'machining', 'wood']
coef.columns = DV_indexes
coef

,chairs,desks,tables
profit,16,20,14
fabrication,4,6,2
assembly,3,8,6
machining,9,6,4
wood,30,40,25


The next cell reads in the RHS of the constraints into their own dataframe. 

In [6]:
rhs = pd.DataFrame(raw_data.iloc[9:16, 6])
rhs

,Unnamed: 6
9,2000
10,2000
11,1440
12,9600
13,300
14,120
15,144


So above we have all the rhs values, but we need to update the indexes to make these easier to keep track of. Note that we have the 4 main constraints and also the 3 ceiling constraints. So - we need to add those ceiling indexes to the ones that are already in the `coef` dataframe - but we don't want `profit` so we start with `[1:]`. Note the use of `extend`. If we use append, it will add the list as a whole inside the list.

In [7]:
idx = list(coef.index[1:])
idx.extend(['chair ceiling', 'desk ceiling', 'table ceiling'])
rhs.index = idx
rhs.columns = ['rhs']
rhs

,rhs
fabrication,2000
assembly,2000
machining,1440
wood,9600
chair ceiling,300
desk ceiling,120
table ceiling,144


## Create pyomo model, decision variables, objective function, and constraints

In this section you will instantiate the decision variables, objective function and constraints for the pyomo model.

Next, instantiate a ConcreteModel and store it in the variable `model`.

In [8]:
model = pe.ConcreteModel()

### Define the Decision Variables

Create the decision variables for the number of Chairs, Desks, and Tables. 

Create a pyomo variable named `x` with domain of nonnegative of real numbers.  Make sure you "attach" this variable to the `model` object. 

In [9]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)

To double check what you entered, use '.pprint()'.

In [10]:
model.x.pprint()

x : Size=3, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    chairs :     0 :  None :  None : False :  True : NonNegativeReals
     desks :     0 :  None :  None : False :  True : NonNegativeReals
    tables :     0 :  None :  None : False :  True : NonNegativeReals


### Define the objective function

The next cell defines the objective function `obj` using the data in the `coef` dataframe.  The expression argument, `expr=...`, shows a shorthand way to represent a sumproduct using a "list comprehension".  The `sense=pe.maximize` argument tells Pyomo that you want to maximize.

In [11]:
model.obj = pe.Objective(expr=sum([coef.loc['profit',i]*model.x[i] for i in DV_indexes]), 
                         sense=pe.maximize)

To Double check what you entered, use 'pprint'.

In [12]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 16*x[chairs] + 20*x[desks] + 14*x[tables]


### Define the constraints

Next we need to define the four constraints using the data in the `coef` and `rhs` dataframes.  Make sure to "attach" each constraint to the `model` object. The constraints use a list comprehension to calculate the LHS sumproduct. 

In [13]:
model.cons_fabrication = pe.Constraint(expr=sum([coef.loc['fabrication', i]*model.x[i] 
                                                 for i in DV_indexes]) <= 
                                       rhs.loc['fabrication', 'rhs'])
model.cons_assembly = pe.Constraint(expr=sum([coef.loc['assembly', i]*model.x[i] 
                                              for i in DV_indexes]) <= 
                                    rhs.loc['assembly', 'rhs'])
model.cons_machining = pe.Constraint(expr=sum([coef.loc['machining', i]*model.x[i] 
                                               for i in DV_indexes]) <= 
                                     rhs.loc['machining', 'rhs'])
model.cons_wood = pe.Constraint(expr=sum([coef.loc['wood', i]*model.x[i] 
                                          for i in DV_indexes]) <= 
                                rhs.loc['wood', 'rhs'])
model.cons_chairs_ceil = pe.Constraint(expr=model.x['chairs'] <= rhs.loc['chair ceiling','rhs'])
model.cons_desks_ceil = pe.Constraint(expr=model.x['desks'] <= rhs.loc['desk ceiling','rhs'])
model.cons_tables_ceil = pe.Constraint(expr=model.x['tables'] <= rhs.loc['table ceiling','rhs'])

Check the constraints. __NOTE:__ Lower and Upper shows the allowable range - the Upper is the RHS - and the Lower is all -Inf so we can tell all of these are <= RHS constraints.

For a final check, let's print the entire model (rather than each separate part obj, decision variables, constraints)

In [14]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'chairs', 'desks', 'tables'}

1 Var Declarations
    x : Size=3, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        chairs :     0 :  None :  None : False :  True : NonNegativeReals
         desks :     0 :  None :  None : False :  True : NonNegativeReals
        tables :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 16*x[chairs] + 20*x[desks] + 14*x[tables]

7 Constraint Declarations
    cons_assembly : Size=1, Index=None, Active=True
        Key  : Lower : Body                                   : Upper  : Active
        None :  -Inf : 3*x[chairs] + 8*x[desks] + 6*x[tables] : 2000.0 :   True
    cons_chairs_ceil : Size=1, Index=None, Acti

### Solve the model

In [15]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp4n0dy23p.glpk.raw
 --wglp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpwkfx1sxf.glpk.glp
 --cpxlp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp2x9wg6pv.pyomo.lp
Reading problem data from '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp2x9wg6pv.pyomo.lp'...
8 rows, 4 columns, 16 non-zeros
54 lines were read
Writing problem data to '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpwkfx1sxf.glpk.glp'...
42 lines were written
GLPK Simplex Optimizer, v4.65
8 rows, 4 columns, 16 non-zeros
Preprocessing...
4 rows, 3 columns, 12 non-zeros
Scaling...
 A: min|aij| =  2.000e+00  max|aij| =  4.000e+01  ratio =  2.000e+01
GM: min|aij| =  6.866e-01  max|aij| =  1.456e+00  ratio =  2.121e+00
EQ: min|aij| =  4.714e-01  max|aij| =  1.000e+00  ratio =  2.121e+00
Constructing initial basis...
Size of triangular part is 4
*     0: obj =  -0.000000

This next cell will help you determine whether the solver found a solution or had an error (like poorly defined constraints, or infeasibility).  Pay particular attention to the "Solver" part of the output.

In [16]:
print(success)


Problem: 
- Name: unknown
  Lower bound: 4672.0
  Upper bound: 4672.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 4
  Number of nonzeros: 16
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.016909122467041016
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



### View the Final Model Obj Function and Dec Variables

The next cell will extract the objective function and optimal solution (assuming the solver found a solution).  

In [17]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

DV = []  # create an empty list to store decision variables
for index in DV_indexes:
    DV.append(round(model.x[index].value, 3))
pd.DataFrame({'DV':DV_indexes,
             'Value':DV})

optimal objective value maximum profit = $4672.00


,DV,Value
0,chairs,16.0
1,desks,120.0
2,tables,144.0


### View the slack

Print the slack in each constraint. 

In [18]:
print(f'Fabrication slack = {model.cons_fabrication.slack()}')
print(f'Assembly slack = {model.cons_assembly.slack()}')
print(f'Machining slack = {model.cons_machining.slack()}')
print(f'Wood slack = {model.cons_wood.slack()}')
print(f'Chairs Ceiling slack = {model.cons_chairs_ceil.slack()}')
print(f'Desks Ceiling slack = {model.cons_desks_ceil.slack()}')
print(f'Tables Ceiling slack = {model.cons_tables_ceil.slack()}')

Fabrication slack = 928.0
Assembly slack = 128.0
Machining slack = 0.0
Wood slack = 720.0
Chairs Ceiling slack = 284.0
Desks Ceiling slack = 0.0
Tables Ceiling slack = 0.0


### View the final LHS values

Also examine the final values for the LHS for each constraint. The "Body" is the final value.

In [ ]:
model.cons_fabrication.display()
model.cons_assembly.display()
model.cons_machining.display()
model.cons_wood.display()
model.cons_chairs_ceil.display()
model.cons_desks_ceil.display()
model.cons_tables_ceil.display()

cons_fabrication : Size=1
    Key  : Lower : Body   : Upper
    None :  None : 1072.0 : 2000.0
cons_assembly : Size=1
    Key  : Lower : Body   : Upper
    None :  None : 1872.0 : 2000.0
cons_machining : Size=1
    Key  : Lower : Body   : Upper
    None :  None : 1440.0 : 1440.0
cons_wood : Size=1
    Key  : Lower : Body   : Upper
    None :  None : 8880.0 : 9600.0
cons_chairs_ceil : Size=1
    Key  : Lower : Body : Upper
    None :  None : 16.0 : 300.0
cons_desks_ceil : Size=1
    Key  : Lower : Body  : Upper
    None :  None : 120.0 : 120.0
cons_tables_ceil : Size=1
    Key  : Lower : Body  : Upper
    None :  None : 144.0 : 144.0


##### [Back to Top](#TOC)

## 3. ERRORS <a id=S3></a>

```
ERROR 1:
#if you get an error WARNING: Could not locate the 'glpsol' executable, which is required for solver 'glpk'
#find where glpsol executable is located and copy path
#To find, search for ipopt or glpsol and find the .exe files on windows or just the names on Mac which are executable

#On my mac, it is the following - copy your path and run the following line
solverpath_glpsol = r"/Users/Kellie/opt/anaconda3/bin/glpsol"

#On my windows machine, it is the following – copy your path and run the following
solverpath_glpsol = r"C:\Users\kelli\anaconda3\Library\bin\glpsol"

# Run the solver.  
#opt = pe.SolverFactory('glpk')
#if you had trouble finding glpsol executable, run the following instead of above
opt = pe.SolverFactory('glpk', executable=solverpath_glpsol)

```

```
ERROR 2:
If you get an error about Implicitly replacing the Component attribute - this means you are rerunning code to define decision variables, constraints or an objective that you have already defined (already run). You have to run the code "from the top" starting with the model = pe.ConcreteModel()
```

##### [Back to Top](#TOC)